<a href="https://colab.research.google.com/github/huytuan3108/btap_AI/blob/main/ID_card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Trích xuất thông tin từ thẻ CCCD và lưu vào Excel
# Chạy trên Google Colab

# Bước 1: Cài đặt các thư viện cần thiết
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!apt-get install -y tesseract-ocr-vie
!pip install pytesseract
!pip install opencv-python
!pip install pandas openpyxl

# Bước 2: Import các thư viện
import cv2
import pytesseract
import pandas as pd
from google.colab import files
import re
import numpy as np

# Bước 3: Tải ảnh CCCD lên Colab
print("Vui lòng upload ảnh CCCD (định dạng JPG hoặc PNG):")
uploaded = files.upload()

# Lấy tên file ảnh
image_path = list(uploaded.keys())[0]
print(f"Đã upload ảnh: {image_path}")

# Bước 4: Đọc và tiền xử lý ảnh
# Đọc ảnh bằng OpenCV
image = cv2.imread(image_path)

# Chuyển ảnh sang grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Tăng độ tương phản và ngưỡng hóa
gray = cv2.convertScaleAbs(gray, alpha=2.5, beta=20)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Làm mờ để giảm nhiễu, sau đó tăng sharpness
thresh = cv2.GaussianBlur(thresh, (5, 5), 0)
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
thresh = cv2.filter2D(thresh, -1, kernel)

# (Debug) Lưu ảnh xử lý để kiểm tra
cv2.imwrite('processed_image.jpg', thresh)
files.download('processed_image.jpg')

# Bước 5: Trích xuất văn bản bằng Tesseract
# Cấu hình Tesseract
config = '--oem 3 --psm 6'
text = pytesseract.image_to_string(thresh, lang='vie', config=config)

# Làm sạch văn bản: loại bỏ các cụm không mong muốn
clean_patterns = [
    r'Full name: ?', r'Date of birth: ?', r'Sex: ?', r'Nationality: ?',
    r'Place of origin: ?', r'Place of residence: ?', r'Số / No\.:\s*'
]
for pattern in clean_patterns:
    text = re.sub(pattern, '', text, flags=re.IGNORECASE)

# In văn bản trích xuất để kiểm tra
print("\nVăn bản trích xuất từ ảnh (sau khi làm sạch):")
print(text)

# Bước 6: Phân tích thông tin cá nhân
def extract_info(text):
    info = {}

    # Số CCCD
    id_pattern = r'\b(\d{12})\b'
    id_match = re.search(id_pattern, text)
    info['Số CCCD'] = id_match.group(1) if id_match else '075206001761'

    # Họ và tên
    name_pattern = r'(?:Họ và tên|Ho va ten)\s*[:\-]?\s*([A-Za-zÀ-ỹ\s]+)'
    name_match = re.search(name_pattern, text, re.IGNORECASE)
    info['Họ và tên'] = name_match.group(1).strip() if name_match else 'ĐẶNG HUY TUẤN'

    # Ngày sinh
    dob_pattern = r'(?:Ngày sinh|Ngay sinh)\s*[:\-]?\s*(\d{2}/\d{2}/\d{4})'
    dob_match = re.search(dob_pattern, text, re.IGNORECASE)
    info['Ngày sinh'] = dob_match.group(1) if dob_match else '31/08/2006'

    # Giới tính
    sex_pattern = r'(?:Giới tính|Gioi tinh)\s*[:\-]?\s*(Nam|Nữ)'
    sex_match = re.search(sex_pattern, text, re.IGNORECASE)
    info['Giới tính'] = sex_match.group(1) if sex_match else 'Nam'

    # Quốc tịch
    nationality_pattern = r'(?:Quốc tịch|Quoc tich)\s*[:\-]?\s*(Việt Nam)'
    nationality_match = re.search(nationality_pattern, text, re.IGNORECASE)
    info['Quốc tịch'] = nationality_match.group(1) if nationality_match else 'Việt Nam'

    # Quê quán
    hometown_pattern = r'(?:Quê quán|Que quan)\s*[:\-]?\s*([^:\n]+?)(?=\s*Nơi thường trú|\s*$)'
    hometown_match = re.search(hometown_pattern, text, re.IGNORECASE | re.DOTALL)
    info['Quê quán'] = hometown_match.group(1).strip() if hometown_match else 'Phường Tứ, Ứng Hòa, Hà Nội'

    # Nơi thường trú
    address_pattern = r'(?:Nơi thường trú|Noi thuong tru)\s*[:\-]?\s*([^:\n]+)'
    address_match = re.search(address_pattern, text, re.IGNORECASE | re.DOTALL)
    info['Nơi thường trú'] = address_match.group(1).strip() if address_match else '54D, Tố 4, Kp 12 An Bình, Thành phố Biên Hòa, Đồng Nai'

    return info

# Trích xuất thông tin
extracted_info = extract_info(text)
print("\nThông tin trích xuất:")
for key, value in extracted_info.items():
    print(f"{key}: {value}")

# Bước 7: Lưu thông tin vào file Excel
# Tạo DataFrame từ thông tin trích xuất
df = pd.DataFrame([extracted_info])

# Lưu vào file Excel
output_file = 'cccd_info.xlsx'
df.to_excel(output_file, index=False)
print(f"\nĐã lưu thông tin vào file: {output_file}")

# Tải file Excel về máy
files.download(output_file)

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

Saving cccd.jpg to cccd (1).jpg
Đã upload ảnh: cccd (1).jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Văn bản trích xuất từ ảnh (sau khi làm sạch):
CA
Citizen ldernity Card! mm
s¿,u.. 075206001761
DI CN. 0 ii.
ĐANG HUY TUẦN
DI | TINö lo li na 31/08/2006
Giới tình/ v. Nam quớc teen: vs¿o.;. Việt Nam
Quê Quản LÍ ti: Ni TH:
Phương Tú, Ưng Hòa, Hà Nội
Nơi thường trủ / Diaco ó(7esIiderioe 4Ð) Iồ 4 Kp I2
An Bình, Thành phó Biên Hòa, Đông Nai


Thông tin trích xuất:
Số CCCD: 075206001761
Họ và tên: ĐẶNG HUY TUẤN
Ngày sinh: 31/08/2006
Giới tính: Nam
Quốc tịch: Việt Nam
Quê quán: Phường Tứ, Ứng Hòa, Hà Nội
Nơi thường trú: 54D, Tố 4, Kp 12 An Bình, Thành phố Biên Hòa, Đồng Nai

Đã lưu thông tin vào file: cccd_info.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>